**Cálculadora de diagrama de cortantes y flectores (P9)**
------------------------------------------------------------------------------
***

> El siguiente código calcula el diagrama de cortante y flector para una barra como la de la figura con cuatro nodos
> $j=1,...,4$ en los que puede haber esfuerzos $P_{j}$, momentos $T_{j}$ y reacciones $R_{jy}$, $M_{jz}$.
>Permite resolver los problemas de la asignatura que no tienen más de 4 puntos con cargas/momentos/reacciones puntuales. Permite añadir cargas distribuidas también.

<img src="disp.png" width="800">



> ¿Cómo se usa? 
- Añadir las solicitaciones (cargas Pj, momentos Tj y cargas distribuidas q_{j,j+1}).
- Añadir las ecuaciones ΣFy=0 y ΣMz=0 en la primera y segunda linea del comando linsolve. Las reacciones que no participen en las ecuaciones (R0y, R1y, R2, R3 y,M0z, M1z, M2z y M3z) hay que añadirlas en las lineas de abajo
>Nota: Todas las reacciones y solicitaciones se consideran positivos en dirección positiva de los ejes cartesianos.

In [1]:
from sympy import *             # Librería para trabajo simbólico
import math                     # Librería para utilizar símbolos matemáticos como el número pi, que se escribe como math.pi
import numpy as np              # Librería para poder trabajar con matrices y vectores
import matplotlib.pyplot as plt # Librería para poder dibujar gráficas

x,a,P,Mo,Ra,Lt=symbols('x a P Mo Ra Lt') #variable simbólica coordenada x
R0y,R1y,R2y,R3y=symbols('R0y R1y R2y R3y')
M0z,M1z,M2z,M3z=symbols('M0z M1z M2z M3z')

A1,B1,A2,B2,A3,B3=symbols('A1 B1 A2 B2 A3 B3')

#solicitaciones
P0=Ra #N  la carga en x=0
P1=-P #N  la carga en x=L1
P2=0.0 #N  la carga en x=L1+L2
P3=0.0 #N  la carga en x=L1+L2+L3

T0=0.0 #Nm  el par en x=0
T1=-Mo #Nm  la par en x=L1
T2=0.0 #Nm  la par en x=L1+L2+L3
T3=0.0 #Nm  la par en x=L1+L2+L3

#Lt=9.0
#cargas distribuidas (positivas en sentido +y)
#W=symbols('W')
#W=1

qq01=0 #N/m la carga distribuida entre los puntos 0 y 1 (entre x=0 y x=L1)
qq12=0 #N/m
qq23=0 #N/m

#geometría
L1=a #longitud tramo 01
L2=Lt-a #longitud tramo 12
L3=0.0 #longitud tramo 23
#... añadir más si hay más tramos

L=L1+L2+L3 #longitud total

#material

E=symbols('E')
I=symbols('I')

#E=1.0e0
#I=1.0e0

#Cálculo estático. Añadir las ecuaciones ΣFy=0 y ΣMz=0 en la primera y segunda linea. Las reacciones que no participen
#en las ecuaciones (R0y,R1y,R2y,M0z,M1z,M2z) hay que añadirlas en las lineas de abajo
#Nota: Todas las reacciones y solicitaciones se consideran positivos en dirección positiva de los ejes cartesianos.
sol,=linsolve([  Ra+R2y+P1,   #se pone todo positivo siempre
               M0z +P1*L1+T1+R2y*(L1+L2),
              R0y,
              R1y,
              R3y,
              M1z,
              M2z,
              M3z,
             ], (R0y,R1y,R2y,R3y,M0z,M1z,M2z,M3z))
print (sol)


#reacciones
R0y=sol[0] #N reacción en y en x=0 
R1y=sol[1] #N reacción en y en x=L1 
R2y=sol[2] #N reacción en y en x=L1+L2 
R3y=sol[3] #N reacción en y en x=L1+L2+L3
M0z=sol[4] #Nm reacción momento en x=0 
M1z=sol[5] #Nm reacción momento en x=L1 
M2z=sol[6] #Nm reacción momento en x=L1+L2 
M3z=sol[7] #Nm reacción momento en x=L1+L2+L3 


#-------------------------------------------------------------------------------------#
#---------------------------------- CÁLCULO TRAMOS -----------------------------------#
#-------------------------------------------------------------------------------------#

#--------------------------------------TRAMO 1----------------------------------------#
#Definimos la carga distribuida
def q1(x):        
    return  qq01

#Definimos el cortante
VL=R0y+P0 #Este es el valor del cortante a la izquierda del tramo 1, lo que llamamos V+(x=0)
def cortante1(x):        
    return  integrate(q1(x), (x, 0, x)) + VL #esto quiere decir que integramos q(x) dandole a x limites entre 0 y x. 

#Definimos el flector
ML=-M0z-T0 #Este es el valor del flector a la izquierda del tramo 1, lo que llamamos M+(x=0)
def flector1(x):        
    return  integrate(cortante1(x), (x, 0, x)) + ML #en MPa 

V1=cortante1(x) 
M1=flector1(x)
print("La expresión del cortante en el tramo 1 es: ", V1)
print("La expresión del flector en el tramo 1 es: ", M1)


#Elástica
def giro1(x,A1):        
    return  integrate(flector1(x)/(E*I),x) + A1 #en MPa

def deflexion1(x,A1,B1):        
    return  integrate(giro1(x,A1),x) + B1 #en MPa

G1=giro1(x,A1) 
U1=deflexion1(x,A1,B1)
print("La expresión del giro en el tramo 1 es: ", G1)
print("La expresión de la deflexion en el tramo 1 es: ", U1)
print("")


#--------------------------------------TRAMO 2----------------------------------------#
#Definimos la carga distribuida
def q2(x):        
    return  qq12

#Definimos el cortante
VL=V1.subs([(x,L1)]) + R1y + P1 #Este es el valor del cortante a la izquierda del tramo 2, lo que llamamos V+(x=L1) que se 
                          #calcula mediante el equilibrio de la rebanada como el cortante a la izquierda V-(x=L1) más
                          #la carga puntual en x=L1
def cortante2(x):        
    return  integrate(q2(x), (x, L1, x)) + VL #esto quiere decir que integramos q(x) dandole a x limites entre 0 y x. 

#Definimos el flector
ML=M1.subs([(x,L1)]) - M1z - T1 #Este es el valor del flector a la izquierda, lo que llamamos M+(x=L1)
def flector2(x):        
    return  factor(integrate(cortante2(x), (x, L1, x)) + ML) #en MPa 

V2=cortante2(x) 
M2=flector2(x)
if L2 !=0:
    print("La expresión del cortante en el tramo 2 es: ", V2)
    print("La expresión del flector en el tramo 2 es: ", M2)

#elástica
def giro2(x):        
    return  simplify(integrate(flector2(x)/(E*I),x) + A2) #en MPa

def deflexion2(x):        
    return  simplify(integrate(giro2(x),x) + B2) #en MPa

G2=giro2(x) 
U2=deflexion2(x)
if L2 !=0:
    print("La expresión del giro en el tramo 2 es: ", G2)
    print("La expresión de la deflexion en el tramo 2 es: ", U2)
    print("",simplify(G2.subs([(x,Lt)])) )



#--------------------------------------TRAMO 3----------------------------------------#
#Definimos la carga distribuida
def q3(x):        
    return  qq23

#Definimos el cortante
VL=V2.subs([(x,L1+L2)]) + R2y + P2 #Este es el valor del cortante a la izquierda del tramo 2, lo que llamamos V+(x=L1) que se 
                          #calcula mediante el equilibrio de la rebanada como el cortante a la izquierda V-(x=L1) más
                          #la carga puntual en x=L1
def cortante3(x):        
    return  integrate(q3(x), (x, L1+L2, x)) + VL #esto quiere decir que integramos q(x) dandole a x limites entre 0 y x. 

#Definimos el flector
ML=M2.subs([(x,L1+L2)]) - M2z - T2 #Este es el valor del flector a la izquierda, lo que llamamos M+(x=L1)
def flector3(x):        
    return  integrate(cortante3(x), (x, L1+L2, x)) + ML #en MPa 

V3=cortante3(x) 
M3=flector3(x)
if L3 !=0:
    print("La expresión del cortante en el tramo 3 es: ", V3)
    print("La expresión del flector en el tramo 3 es: ", M3)

#Elástica
def giro3(x):        
    return  integrate(flector3(x)/(E*I),x) + A3 #en MPa

def deflexion3(x):        
    return  integrate(giro3(x),x) + B3 #en MPa

G3=giro3(x) 
U3=deflexion3(x)
if L3 !=0:
    print("La expresión del giro en el tramo 3 es: ", G3)
    print("La expresión de la deflexion en el tramo 3 es: ", U3)
    print("")

    
#-------------------------------------------------------------------------------------#
#----------------------------------- C.C. ELÁSTICA -----------------------------------#
#-------------------------------------------------------------------------------------#    
  
G1_0=G1.subs([(x,0)])    #giro en tramo 1, punto 0
G1_1=G1.subs([(x,L1)])   #giro en tramo 1, punto 1
G2_1=G2.subs([(x,L1)])   #giro en tramo 2, punto 1
G2_2=G2.subs([(x,L1+L2)]) 
G3_2=G3.subs([(x,L1+L2)])    
G3_3=G3.subs([(x,L1+L2+L3)]) 

U1_0=U1.subs([(x,0)])    #deflexion en tramo 1, punto 0
U1_1=U1.subs([(x,L1)])   #deflexion en tramo 1, punto 1
U2_1=U2.subs([(x,L1)])   #deflexion en tramo 2, punto 1
U2_2=U2.subs([(x,L1+L2)]) 
U3_2=U3.subs([(x,L1+L2)])    
U3_3=U3.subs([(x,L1+L2+L3)]) 

    
#ejemplo CC en linsolve para una barra con dos tramos, a la izquierda apoyada y a la derecha empotrada:
# sole,=linsolve([    U1_0,      #deflexion en tramo1, punto 0 es nula (apoyo)
#                     U1_1-U2_1, #deflexion en tramo1, punto 1 es igual a deflexion en tramo2, punto 1 (continuidad)
#                     G1_1-G2_1, #giro en tramo1, punto 1 es igual a giro en tramo2, punto 1 (continuidad deriv.)
#                     G2_2       #giro en tramo2, punto 2 es nula (apoyo)
#              ], (A1,B1,A2,B2)) #incognitas
    
sole,=linsolve([  G1_0,
                  U1_0,
                  U2_2,
                  G1_1-G2_1,
                  U1_1-U2_1
             ], (A1,B1,B2,A2,Ra))

#print (sole)
print("")
print("")
print("Ra es:", factor(sole[4]))
print("")

G1_exp=simplify(G1.subs([(A1,sole[0]),(Ra,sole[4])]))
U1_exp=simplify(U1.subs([(A1,sole[0]),(B1,sole[1]),(Ra,sole[4])]))
print("Aplicadas las condiciones de contorno...")
print("La expresión del giro en el tramo 1 es: ", G1_exp)
print("La expresión de la deflexion en el tramo 1 es: ", U1_exp)
print("")

if L2 !=0:
    G2_exp=simplify(G2.subs([(A2,sole[2]),(Ra,sole[4])]))
    U2_exp=simplify(U2.subs([(A2,sole[2]),(B2,sole[3]),(Ra,sole[4])]))
    print("La expresión del giro en el tramo 2 es: ", G2_exp)
    print("La expresión de la deflexion en el tramo 2 es: ", U2_exp)
    print("")

    
if L3 !=0:
    G3_exp=G3.subs([(A3,sole[4])])
    U3_exp=U3.subs([(A3,sole[4]),(B3,sole[5])])
    print("La expresión del giro en el tramo 3 es: ", G3_exp)
    print("La expresión de la deflexion en el tramo 3 es: ", U3_exp)
    print("")

    
    
#calculo superposicion

Rb=symbols('Rb') 

d1= -Rb*Lt**3/(3*E*I)
d2= P*a**2/(6*E*I)*(3*Lt-a)
d3= Mo*a/(2*E*I)*(2*Lt-a)

sole,=linsolve([  d1+d2+d3,
             ], (Rb))

print("Rb es:", factor((sole[0])))
print("Ra es:", factor(P-(sole[0])))


(0, 0, P - Ra, 0, -Lt*(P - Ra) + Mo + P*a, 0, 0, 0)
La expresión del cortante en el tramo 1 es:  Ra
La expresión del flector en el tramo 1 es:  Lt*(P - Ra) - Mo - P*a + Ra*x
La expresión del giro en el tramo 1 es:  A1 + Ra*x**2/(2*E*I) + x*(Lt*P - Lt*Ra - Mo - P*a)/(E*I)
La expresión de la deflexion en el tramo 1 es:  A1*x + B1 + Ra*x**3/(6*E*I) + x**2*(Lt*P - Lt*Ra - Mo - P*a)/(2*E*I)

La expresión del cortante en el tramo 2 es:  -P + Ra
La expresión del flector en el tramo 2 es:  -(-Lt + x)*(P - Ra)
La expresión del giro en el tramo 2 es:  (A2*E*I + Lt*x*(P - Ra) + x**2*(-P + Ra)/2)/(E*I)
La expresión de la deflexion en el tramo 2 es:  (E*I*(A2*x + B2) + Lt*x**2*(P - Ra)/2 + x**3*(-P + Ra)/6)/(E*I)
 (A2*E*I + Lt**2*(P - Ra)/2)/(E*I)


Ra es: (2*Lt**3*P - 6*Lt*Mo*a - 3*Lt*P*a**2 + 3*Mo*a**2 + P*a**3)/(2*Lt**3)

Aplicadas las condiciones de contorno...
La expresión del giro en el tramo 1 es:  x*(2*Lt*(-2*Lt**3*P + 2*Lt**2*(Lt*P - Mo - P*a) + 3*Mo*a**2 + 2*P*a**3 + 3*a*(Lt - a)*(2*Mo + 